In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack

from stable_baselines3.common.evaluation import evaluate_policy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

# Environment

Create a very simple custom environment to test our understanding of:
* Steps
* Epsiodes

In [2]:
from stable_baselines3.common.env_checker import check_env

In [3]:
class EnvSimpleFixedDuration(Env):
    listActionTemperature = (100, 130, 150)
    listObservationTemperature = (100, 130, 150)

    def __init__(self):
        self.action_space = Discrete(3)
        #self.observation_space =  Discrete(3)
        self.observation_space = Box(low=np.array([90]), high=np.array([200]))
        print("Init")
        self.isVerbose = False
        self.reset()

    def setVerbose(self):
        self.isVerbose = True

    def step(self, action):
        temperatureAction = self.listActionTemperature[action]

        self.state = temperatureAction
        self.stepCurrent += 1

        if (self.state == 130):
            reward = 1
        else:
            reward = 0

        done = self.stepCurrent > 3

        #obs = self.listObservationTemperature.index(self.state)
        obs = np.array([self.state], dtype=np.float32)

        #info = self.listObservationTemperature[obs]
        info = {}

        if(self.isVerbose):
            print("....Step: " + str(self.stepCurrent))
            print("......Observation: " + str(obs))
            print("......     Reward: " + str(reward))
            print("......       Info: " + str(info))

        return obs, reward, done, info

    def render(self):
        pass
    
    def reset(self):
        self.state = 100
        self.stepCurrent = 0

        if(self.isVerbose):
            print("..Reset")
            
        return np.array([self.state], dtype=np.float32)
env = EnvSimpleFixedDuration()
check_env(env, warn=True)

Init


e:\anaconda3\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


# Test the environment

In [4]:
env = EnvSimpleFixedDuration()

episodeMax = 3
for episode in range(1, episodeMax+1):
    print("Episode: " + str(episode))
    state = env.reset()
    done = False
    rewardTotal = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        rewardTotal += reward
    print(".Score: " + str(rewardTotal))
env.close()

Init
Episode: 1
.Score: 1
Episode: 2
.Score: 0
Episode: 3
.Score: 2


# Training

In [5]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [8]:
model.learn(total_timesteps=12000)

Logging to Training\Logs\PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4        |
|    ep_rew_mean     | 1.81     |
| time/              |          |
|    fps             | 1606     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 4           |
|    ep_rew_mean          | 2.43        |
| time/                   |             |
|    fps                  | 1098        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.023139067 |
|    clip_fraction        | 0.769       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.975      |
|    explained_variance   | 0.0277      |

In [7]:
!tensorboard --logdir={log_path}

^C
